In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
import os
from itertools import product
import math
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import time

sys.path.append(os.path.abspath("../"))

import scipy.stats

folder = "/content/drive/MyDrive/UKP/mnist_experiments/distances/widthdepth/5000_train/"

# loading test distance file and seeing its entries
data = np.load(f"{folder}/width100_depth2_seed2_rep_width100_depth4_seed2_rep.npz")
print(data.files)

distnames = data.files

reps = []
reps_folder = f"/content/drive/MyDrive/UKP/mnist_experiments/reps/train/5000_eval"
filenames = os.listdir(reps_folder)
for filename in filenames:
  if "saved" not in filename and "seed2" in filename and "depth3" not in filename and "depth6" not in filename and "depth8":
    reps.append(filename[:-4])


reps = np.sort(reps)

model_names = reps
num_models = len(model_names)

In [ ]:
# Code to extract each different distance into its own array
dist_array = {distname: [] for distname in distnames}
# dist_array = {distname: np.array([]) for distname in distnames}
distfilenames = np.sort(os.listdir(folder))

total_distfilenames = len(distfilenames)
print(f"Total number of distance files: {total_distfilenames}")

In [ ]:
def extract_widthdepth(modelname):
  parts = modelname.split("_")
  x = []
  for i in range(2):
    x.append(int(''.join(filter(str.isdigit,parts[i]))))
  return x[1], x[0]

model_names = sorted(model_names, key=extract_widthdepth)


In [ ]:
for i in range(len(model_names)-1):
  first_model_name = model_names[i]
  for j in range(i+1,len(model_names)):
    second_model_name = model_names[j]
    check_string1 = f"{first_model_name}_{second_model_name}.npz"
    check_string2 = f"{second_model_name}_{first_model_name}.npz"
    print(f"Checking File ({i} {first_model_name},{j} {second_model_name})")

    for filename in distfilenames:
      if filename == check_string1 or filename == check_string2:
        file_path = os.path.join(folder, filename)
        data = np.load(file_path)

        for distname in distnames:
            if distname in data:
                dist_array[distname].append(data[distname])
                # dist_array[distname] = np.append(dist_array[distname],data[distname])

In [ ]:
for distname in distnames:
  if "GULP" not in distname and "UKP" not in distname:
    dist_array[distname] = np.array(dist_array[distname])
    # print("Nothing to do")
  else:
    dist_array[distname] = np.sqrt(np.array(dist_array[distname]))
    # dist_array[distname] = np.sqrt(dist_array[distname])
    print(f"distname = {distname}, length = {len(dist_array[distname])}")

In [ ]:
np.savez(f"/content/drive/MyDrive/UKP/mnist_experiments/distances/widthdepth/5000_train_categorized/all_distances_categorized_correctedfinal.npz", **dist_array)